In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn import neighbors

# Convert predicted scores to recommendations

We need to suppress things that have either been purchased or added to the cart by the given user previously

In [3]:
events = pd.read_csv('../data/retailrocket-recommender-system-dataset/events.csv')
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [42]:
suppress = events.loc[
    events.event.isin('transaction addtocart'.split()),
    'visitorid itemid'.split()
]
suppress.head()

,visitorid,itemid
17,287857,5206
19,158090,10572
63,1193904,255275
112,599528,356475
130,599528,356475


In [43]:
suppress = suppress.groupby('visitorid itemid'.split()).count()
suppress['dropcol'] = True
suppress.head()

dropcol
visitorid itemid         
6         65273      True
150       452955     True
155       41882      True
          181405     True
          224623     True

In [52]:
scores = pd.read_csv('../data/retailrocket-recommender-system-dataset/pred-test-scores.csv')
scores.head()

,visitorid,itemid,score
0,1264934,12836,0.48
1,1264934,28367,0.64
2,1264934,36904,0.68
3,1264934,46232,0.64
4,1264934,48557,0.44


In [53]:
merged = pd.merge(scores, suppress, left_on='visitorid itemid'.split(), right_index=True, how='left')

In [54]:
merged.shape

(4801723, 4)

In [55]:
suppress.shape

(64250, 1)

In [56]:
recs.shape

(4801723, 3)

In [68]:
recs = merged[~merged.dropcol.fillna(False)]
recs = recs.drop('dropcol', axis=1)
recs.head()

,visitorid,itemid,score
0,1264934,12836,0.48
1,1264934,28367,0.64
2,1264934,36904,0.68
3,1264934,46232,0.64
4,1264934,48557,0.44


In [69]:
gr = recs.sort_values('visitorid score'.split(), ascending=False).groupby('visitorid')

In [81]:
df_recs = pd.concat(
    [gr.nth(i) for i in range(4)],
    axis=1
)
df_recs.head()

,itemid,score,itemid,score,itemid,score,itemid,score
visitorid,,,,,,,,
1264934,431417,6.40,184011.0,5.64,339822.0,5.32,374855.0,5.32
1264938,306285,1.12,454467.0,1.12,125084.0,0.92,147555.0,0.92
1264939,454467,1.12,125084.0,0.92,147555.0,0.92,432748.0,0.88
1264942,119736,6.32,320130.0,2.96,57723.0,1.80,420960.0,1.52
1264943,198209,1.40,37254.0,1.12,454467.0,1.12,411886.0,1.04


In [83]:
df_recs.columns = pd.MultiIndex.from_product([
    range(4),
    'itemid score'.split()
])
df_recs.head()

0               1               2               3      
           itemid score    itemid score    itemid score    itemid score
visitorid                                                              
1264934    431417  6.40  184011.0  5.64  339822.0  5.32  374855.0  5.32
1264938    306285  1.12  454467.0  1.12  125084.0  0.92  147555.0  0.92
1264939    454467  1.12  125084.0  0.92  147555.0  0.92  432748.0  0.88
1264942    119736  6.32  320130.0  2.96   57723.0  1.80  420960.0  1.52
1264943    198209  1.40   37254.0  1.12  454467.0  1.12  411886.0  1.04

In [84]:
df_recs.to_csv('../data/retailrocket-recommender-system-dataset/test-recommendations.csv')

In [85]:
!head ../data/retailrocket-recommender-system-dataset/test-recommendations.csv

,0,0,1,1,2,2,3,3
,itemid,score,itemid,score,itemid,score,itemid,score
visitorid,,,,,,,,
1264934,431417,6.4,184011.0,5.64,339822.0,5.32,374855.0,5.32
1264938,306285,1.12,454467.0,1.12,125084.0,0.92,147555.0,0.92
1264939,454467,1.12,125084.0,0.92,147555.0,0.92,432748.0,0.88
1264942,119736,6.32,320130.0,2.96,57723.0,1.8,420960.0,1.52
1264943,198209,1.4,37254.0,1.12,454467.0,1.12,411886.0,1.04
1264944,454467,1.12,125084.0,0.92,147555.0,0.92,432748.0,0.88
1264946,239514,1.92,454467.0,1.12,17137.0,1.0,125084.0,0.92
